# Prepare Dictionary

In [ ]:
# Select the dictionary.
# RAE = Spanish Dictionary words
# SQL = data scheme dictionary
# ALL = both dictionaries
DICTIONARY = 'SQL'

In [ ]:
import os

d_rae = False
d_sql = False
dict_rae = dict_sql = None
if DICTIONARY == 'RAE':
  dict_rae = True
elif DICTIONARY == 'SQL':
  dict_sql = True
elif DICTIONARY == 'ALL':
  dict_rae = dict_sql = True
else:
  print("Not a valid dictionary")
  raise

dic = open('dict.txt', 'w')
if dict_rae:
  !cp '/content/drive/MyDrive/TFG/TextRepair/dict_rae_txt.zip' '.'
  !unzip dict_rae_txt.zip
  for filename in os.listdir('./dics'):
    with open(os.path.join('./dics', filename), 'r') as f:
      for l in f.readlines():
        line = l[:-1].split(', ')
        dic.write(line[0] + '\n')
        # gender variants of the word
        if len(line) > 1:
          for i in range(1, len(line)):
            n = len(line[i])
            dic.write(line[0][:-n] + line[i] + '\n')
        
if dict_sql:
  !cp '/content/drive/MyDrive/TFG/HTG/input/PerfectSQL_2.sql' .
  with open('PerfectSQL_2.sql', 'r') as f:
    fi = f.read().split()
    for word in fi:
      dic.write(word + '\n')

# Words repair definition

In [ ]:
import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('dict.txt').read()))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnñopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

# Repair the predictions

In [6]:
from tqdm import tqdm
!cp '/content/drive/MyDrive/TFG/SimpleHTR/SIM_BeamSearch/predictions.out' '.'

with open('predictions_fixed.out', 'w') as fix:
  with open('predictions.out', 'r') as p:
    for line in tqdm(p.readlines()):
      L = line.split()
      if L[0] == '###':
        fix.write(line)
        continue
      repaired_L = ""
      for word in L:
        repaired_w = correction(word)
        repaired_L += " " + repaired_w
      fix.write(repaired_L + '\n')

100%|██████████| 2676/2676 [1:01:11<00:00,  1.37s/it]


In [7]:
!cp 'predictions_fixed.out' '/content/drive/MyDrive/TFG/SimpleHTR/SIM_BeamSearch/spellingcorrector/predictions_fixed_SQL.out'